In [119]:
import json
import os



In [120]:
# Load JSON
# Use absolute path to ensure it works regardless of working directory
json_path = '/Users/yashvibhagat/large_exper_unstructured_data/3_combine_jsons_together/combined_papers.json'

with open(json_path, 'r') as f:
    data = json.load(f)

# print(f"Successfully loaded data. Number of papers: {len(data.get('papers', []))}")


In [121]:
# Count total number of curves across all papers and graphs
total_curves = 0
for paper in data.get('papers', []):
    for graph in paper.get('graphs', []):
        total_curves += len(graph.get('curves', []))

print(f"Total number of curves in the combined JSON file: {total_curves}")


Total number of curves in the combined JSON file: 392


In [122]:

# Flatten the nested JSON structure into a list of curves
curves_list = []

# Loop through all papers
for paper in data.get('papers', []):
    # For each paper, loop through all graphs
    for graph in paper.get('graphs', []):
        # Get axis labels WITH units
        x_axis_label_full = graph.get('x_axis_label', '')
        y_axis_label_full = graph.get('y_axis_label', '')
        
        # Extract y-axis unit
        y_unit = 'unknown'
        if 'MPa' in y_axis_label_full:
            y_unit = 'MPa'
        elif 'GPa' in y_axis_label_full:
            y_unit = 'GPa'
        
        # For each graph, loop through all curves
        for curve in graph.get('curves', []):
            # Extract Kocks-Mecking hardening parameters
            km_params = curve.get('Kocks–Mecking_hardening_parameters', {})
            
            # Create a dictionary with all information for this curve
            curve_info = {
                'paper_title': paper.get('paper_title', ''),
                'graph_id': graph.get('graph_id', ''),
                'curve_id': curve.get('curve_id', ''),
                'curve_label': curve.get('curve_label', ''),
                'x_axis_label': x_axis_label_full,
                'y_axis_label': y_axis_label_full,
                'y_unit': y_unit,
                'alloy_composition': curve.get('alloy_composition_in_percentages_corresponding_to_this_curve', {}),
                'curve_data': curve.get('curve_raw_data', {}).get('data', []),
                # Kocks-Mecking Parameters
                'km_theta0_MPa': km_params.get('theta0_MPa', None),
                'km_sigma_sat_MPa': km_params.get('sigma_sat_MPa', None),
                'km_fit_strain_range_min': km_params.get('fit_strain_range', [None, None])[0] if km_params.get('fit_strain_range') else None,
                'km_fit_strain_range_max': km_params.get('fit_strain_range', [None, None])[1] if km_params.get('fit_strain_range') else None,
                'km_savgol_window_points': km_params.get('savgol_filter', {}).get('window_points', None),
                'km_savgol_poly_order': km_params.get('savgol_filter', {}).get('poly_order', None),
                'km_goodness_of_fit_R2': km_params.get('goodness_of_fit_R2', None)
            }
            curves_list.append(curve_info)

print(f"✓ Extracted {len(curves_list)} curves")
print(f"✓ Sample curve keys: {list(curves_list[0].keys()) if curves_list else 'No curves found'}")

✓ Extracted 392 curves
✓ Sample curve keys: ['paper_title', 'graph_id', 'curve_id', 'curve_label', 'x_axis_label', 'y_axis_label', 'y_unit', 'alloy_composition', 'curve_data', 'km_theta0_MPa', 'km_sigma_sat_MPa', 'km_fit_strain_range_min', 'km_fit_strain_range_max', 'km_savgol_window_points', 'km_savgol_poly_order', 'km_goodness_of_fit_R2']


In [123]:
# STEP 2: Convert stress values from GPa to MPa for consistency
# This ensures all stress values are in the same unit (MPa) for future processing

converted_count = 0
unchanged_count = 0
unknown_unit_count = 0

for curve_info in curves_list:
    y_unit = curve_info.get('y_unit', 'unknown')
    curve_data = curve_info.get('curve_data', [])
    
    if y_unit == 'GPa':
        # Convert all y-values from GPa to MPa (multiply by 1000)
        for data_point in curve_data:
            if 'y' in data_point:
                data_point['y'] = data_point['y'] * 1000
        
        # Update the unit label
        curve_info['y_unit'] = 'MPa'
        
        # Update the y_axis_label to reflect MPa (replace GPa with MPa)
        original_label = curve_info.get('y_axis_label', '')
        if 'GPa' in original_label:
            curve_info['y_axis_label'] = original_label.replace('GPa', 'MPa')
        
        converted_count += 1
        
    elif y_unit == 'MPa':
        # Already in MPa, no conversion needed
        unchanged_count += 1
        
    else:
        # Unknown unit - might need manual review
        unknown_unit_count += 1

print("="*60)
print(" Unit Conversion")
print("="*60)
print(f"✓ Converted from GPa to MPa: {converted_count} curves")
print(f"✓ Already in MPa (unchanged): {unchanged_count} curves")
print(f"⚠️  Unknown unit (may need review): {unknown_unit_count} curves")
print(f"\nTotal curves processed: {len(curves_list)}")

# # Show sample of converted curve
# if converted_count > 0:
#     # Find first converted curve to show example
#     for curve_info in curves_list:
#         if 'GPa' not in curve_info.get('y_axis_label', '') and curve_info.get('y_unit') == 'MPa':
            # This was likely converted (or already in MPa)
            # sample_data = curve_info.get('curve_data', [])[:3]  # First 3 points
            # print(f"\n📊 Sample curve (y-axis in MPa):")
            # print(f"  Curve ID: {curve_info['curve_id']}")
            # print(f"  Y-axis label: {curve_info['y_axis_label']}")
            # print(f"  Y-axis unit: {curve_info['y_unit']}")
            # print(f"  Sample data points (first 3):")
            # for i, point in enumerate(sample_data):
            #     print(f"    Point {i+1}: x={point['x']:.4f}, y={point['y']:.2f} MPa")
            # break

print("\n✓ All stress values are now standardized to MPa!")


 Unit Conversion
✓ Converted from GPa to MPa: 7 curves
✓ Already in MPa (unchanged): 385 curves
⚠️  Unknown unit (may need review): 0 curves

Total curves processed: 392

✓ All stress values are now standardized to MPa!


In [124]:
# STEP 3: Standardize alloy composition names (remove _at%, (at%), (wt%), etc.)
# This step happens after GPa to MPa conversion and before wt% to at% conversion

print("="*60)
print("🔄 Step 3: Standardizing Element Names (remove unit suffixes)")
print("="*60)

def get_base_element(key):
    """
    Extract base element name from key, removing all unit indicators.
    Examples:
    - 'Al' -> 'Al'
    - 'Al_at%' -> 'Al'
    - 'Al (at%)' -> 'Al'
    - 'Al (at.%), if some wt.%)' -> 'Al'
    - 'Zr (wt.%)' -> 'Zr'
    - 'Co_at%' -> 'Co'
    - 'Fe (wt%)' -> 'Fe'
    """
    import re
    
    # Remove everything after '(' including the parenthesis
    # This handles: 'Al (at%)', 'Al (at.%), if some wt.%)', 'Zr (wt.%)'
    base = re.sub(r'\s*\(.*$', '', key)
    
    # Remove _at% and _wt% suffixes
    base = base.replace('_at%', '').replace('_wt%', '')
    base = base.replace('_at', '').replace('_wt', '')
    
    # Remove any trailing whitespace
    base = base.strip()
    
    return base

standardized_count = 0
keys_renamed = 0
total_keys_before = 0
total_keys_after = 0

# Track what was renamed for reporting
rename_examples = {}

for curve_info in curves_list:
    composition = curve_info.get('alloy_composition', {})
    
    if not composition:
        continue
    
    total_keys_before += len(composition)
    
    # Standardize element names
    standardized_composition = {}
    
    for key, value in composition.items():
        base_element = get_base_element(key)
        
        # Track if we renamed this key
        if key != base_element:
            keys_renamed += 1
            # Store example of what was renamed
            if base_element not in rename_examples:
                rename_examples[base_element] = []
            if key not in rename_examples[base_element]:
                rename_examples[base_element].append(key)
        
        standardized_composition[base_element] = value
    
    # Update composition with standardized version
    curve_info['alloy_composition'] = standardized_composition
    
    total_keys_after += len(standardized_composition)
    
    if len(standardized_composition) > 0:
        standardized_count += 1

print(f"✓ Standardized compositions: {standardized_count} curves")
print(f"✓ Total keys before: {total_keys_before}")
print(f"✓ Total keys after: {total_keys_after}")
print(f"✓ Keys renamed: {keys_renamed}")

# Show what was renamed
if rename_examples:
    print("\n" + "="*60)
    print("🔄 Renaming Examples:")
    print("="*60)
    for base_element, original_keys in sorted(rename_examples.items())[:10]:
        print(f"  {base_element}:")
        for orig in original_keys[:3]:  # Show first 3 examples
            print(f"    '{orig}' → '{base_element}'")

# Collect all unique column names after standardization
all_column_names = set()
for curve_info in curves_list:
    composition = curve_info.get('alloy_composition', {})
    if composition:
        all_column_names.update(composition.keys())

print("\n" + "="*60)
print(f"📋 All Unique Alloy Composition Column Names ({len(all_column_names)} total)")
print("="*60)
print(sorted(all_column_names))

print("\n✓ Standardization complete!")


🔄 Step 3: Standardizing Element Names (remove unit suffixes)
✓ Standardized compositions: 388 curves
✓ Total keys before: 1971
✓ Total keys after: 1971
✓ Keys renamed: 706

🔄 Renaming Examples:
  Al:
    'Al (at.%)' → 'Al'
    'Al (at%)' → 'Al'
    'Al_at%' → 'Al'
  B:
    'B_at%' → 'B'
  C:
    'C_at%' → 'C'
  Co:
    'Co (at.%)' → 'Co'
    'Co_at%' → 'Co'
    'Co (at%)' → 'Co'
  Cr:
    'Cr (at.%)' → 'Cr'
    'Cr_at%' → 'Cr'
    'Cr (at%)' → 'Cr'
  Cu:
    'Cu (at.%)' → 'Cu'
    'Cu (at%)' → 'Cu'
  Fe:
    'Fe (at.%)' → 'Fe'
    'Fe_at%' → 'Fe'
    'Fe (at%)' → 'Fe'
  Hf:
    'Hf_at%' → 'Hf'
  Mg:
    'Mg (wt.%)' → 'Mg'
  Mn:
    'Mn (at.%)' → 'Mn'
    'Mn_at%' → 'Mn'

📋 All Unique Alloy Composition Column Names (30 total)
['Al', 'B', 'C', 'Co', 'Cr', 'Cu', 'Fe', 'Hf', 'Mg', 'Mn', 'Mo', 'N', 'Nb', 'NbPlusTa', 'Nb_Ta', 'Nd', 'Ni', 'Si', 'Ta', 'Ti', 'V', 'W', 'Y', 'Zr', '_note', 'basis', 'nominal', 'note', 'unit', 'units']

✓ Standardization complete!


In [125]:
# STEP 4: Remove unnecessary columns from alloy compositions
# This happens after standardization

# Columns to remove
COLUMNS_TO_REMOVE = ['NbPlusTa', 'Nb_Ta', '_note', 'basis', 'nominal', 'note', 'unit', 'units']

print("="*60)
print("Step 4: Removing Unnecessary Columns")
print("="*60)

total_removed = 0
curves_affected = 0

for curve_info in curves_list:
    raw_composition = curve_info.get('alloy_composition', {})
    
    if not raw_composition:
        continue
    
    # Create cleaned composition without unwanted columns
    cleaned_composition = {}
    removed_count = 0
    
    for key, value in raw_composition.items():
        if key not in COLUMNS_TO_REMOVE:
            cleaned_composition[key] = value
        else:
            removed_count += 1
    
    # Update the composition
    curve_info['alloy_composition'] = cleaned_composition
    
    if removed_count > 0:
        curves_affected += 1
        total_removed += removed_count

print(f"✓ Removed {total_removed} unwanted column entries")
print(f"✓ Affected {curves_affected} curves")
print(f"\nTotal curves processed: {len(curves_list)}")



Step 4: Removing Unnecessary Columns
✓ Removed 54 unwanted column entries
✓ Affected 51 curves

Total curves processed: 392


In [126]:
# STEP 5: Convert wt% to at%

import re

# Standard atomic weights (g/mol) for common elements in high-entropy alloys
ATOMIC_WEIGHTS = {
    'Al': 26.982,
    'Co': 58.933,
    'Cr': 51.996,
    'Cu': 63.546,
    'Fe': 55.845,
    'Mn': 54.938,
    'Mo': 95.95,
    'Nb': 92.906,
    'Ni': 58.693,
    'Ta': 180.948,
    'Ti': 47.867,
    'V': 50.942,
    'W': 183.84,
    'Zr': 91.224,
    'C': 12.011,
    'N': 14.007,
    'O': 15.999,
    'Si': 28.085,
    'Mg': 24.305,
    'Zn': 65.38,
    'Sn': 118.71,
    'Hf': 178.49,
    'Re': 186.207,
    'Ru': 101.07,
    'Pd': 106.42,
    'Pt': 195.084,
    'Au': 196.967,
    'Ag': 107.868,
    'B': 10.81,
    'P': 30.974,
    'S': 32.06,
}

def extract_numeric_value(value):
    """
    Extract numeric value from string, handling various formats:
    - '≥10 (locally enriched due to added powders)' -> 10.0
    - '5-10' -> 7.5 (average)
    - '~5' -> 5.0
    - '<0.1' -> 0.1
    - 'balance' -> None
    """
    if value is None:
        return None
    
    # If already a number, return it
    if isinstance(value, (int, float)):
        return float(value)
    
    # Convert to string
    value_str = str(value).strip().lower()
    
    # Handle special cases
    if value_str in ['balance', 'bal', 'bal.', 'remainder', 'rem', 'rest']:
        return None  # Will calculate balance later
    
    if value_str in ['trace', 'traces', '-', '', 'n/a', 'na']:
        return 0.0
    
    # Extract all numbers from the string using regex
    numbers = re.findall(r'[-+]?\d*\.?\d+', value_str)
    
    if not numbers:
        return None
    
    # Convert to floats
    numbers = [float(n) for n in numbers]
    
    # Handle ranges (e.g., "5-10" or "5 to 10")
    if len(numbers) >= 2:
        # Take average of range
        return sum(numbers) / len(numbers)
    
    # Single number
    if len(numbers) == 1:
        return numbers[0]
    
    return None

def extract_element_from_key(key):
    """
    Extract the element symbol from a key.
    Examples: 'Al_wt%' -> 'Al', 'Cr(wt%)' -> 'Cr', 'Fe' -> 'Fe'
    """
    # Remove unit indicators
    clean = key.replace('_wt%', '').replace('_at%', '').replace('_wt', '').replace('_at', '')
    clean = clean.replace('(wt%)', '').replace('(at%)', '').replace('(wt)', '').replace('(at)', '')
    clean = clean.replace(' wt%', '').replace(' at%', '').replace(' wt', '').replace(' at', '')
    clean = clean.replace('wt%', '').replace('at%', '')
    clean = clean.replace('(', '').replace(')', '')
    clean = clean.strip()
    return clean

def is_wt_percent_key(key):
    """Check if key indicates weight percent"""
    key_lower = key.lower()
    return 'wt%' in key_lower or 'wt %' in key_lower or '(wt)' in key_lower or '_wt' in key_lower

def is_at_percent_key(key):
    """Check if key indicates atomic percent"""
    key_lower = key.lower()
    return 'at%' in key_lower or 'at %' in key_lower or '(at)' in key_lower or '_at' in key_lower

def convert_wt_to_at(composition_wt_dict):
    """
    Convert weight percent to atomic percent.
    
    Formula:
    at% = (wt% / atomic_weight) / sum(wt% / atomic_weight for all elements) * 100
    
    Args:
        composition_wt_dict: dict with {element: wt%}
    
    Returns:
        dict with {element: at%}
    """
    if not composition_wt_dict:
        return {}
    
    # Calculate mole fractions
    mole_fractions = {}
    total_moles = 0
    balance_element = None
    
    for element, wt_percent in composition_wt_dict.items():
        if wt_percent is None:
            balance_element = element
            continue
        
        if element in ATOMIC_WEIGHTS and wt_percent > 0:
            moles = wt_percent / ATOMIC_WEIGHTS[element]
            mole_fractions[element] = moles
            total_moles += moles
    
    # Handle balance element
    if balance_element and balance_element in ATOMIC_WEIGHTS:
        total_wt = sum(v for v in composition_wt_dict.values() if v is not None)
        if total_wt < 100:
            balance_wt = 100 - total_wt
            moles = balance_wt / ATOMIC_WEIGHTS[balance_element]
            mole_fractions[balance_element] = moles
            total_moles += moles
    
    # Convert to atomic percent
    composition_at = {}
    if total_moles > 0:
        for element, moles in mole_fractions.items():
            composition_at[element] = (moles / total_moles) * 100
    
    return composition_at

print("="*60)
print("🔬 Step 5: Converting wt% to at%")
print("="*60)

converted_count = 0
already_at_count = 0
mixed_units_count = 0
parse_errors = []

for idx, curve_info in enumerate(curves_list):
    composition = curve_info.get('alloy_composition', {})
    
    if not composition:
        continue
    
    try:
        # Separate by unit type
        wt_percent_data = {}
        at_percent_data = {}
        unknown_data = {}
        
        for key, value in composition.items():
            element = extract_element_from_key(key)
            
            # Skip if element not recognized
            if not element:
                continue
            
            # Parse the value
            numeric_value = extract_numeric_value(value)
            
            # Classify by unit type
            if is_at_percent_key(key):
                at_percent_data[element] = numeric_value
            elif is_wt_percent_key(key):
                wt_percent_data[element] = numeric_value
            else:
                # No unit specified - assume wt% (common convention)
                unknown_data[element] = numeric_value
        
        # Determine what conversion is needed
        has_wt = len(wt_percent_data) > 0 or len(unknown_data) > 0
        has_at = len(at_percent_data) > 0
        
        # Convert wt% to at%
        converted_composition = {}
        
        if has_wt and not has_at:
            # All wt%, convert everything
            all_wt = {**wt_percent_data, **unknown_data}
            converted_composition = convert_wt_to_at(all_wt)
            converted_count += 1
            
        elif has_at and not has_wt:
            # All at%, no conversion needed
            converted_composition = at_percent_data
            already_at_count += 1
            
        elif has_wt and has_at:
            # Mixed units - convert wt% and combine with at%
            wt_converted = convert_wt_to_at({**wt_percent_data, **unknown_data})
            converted_composition = {**at_percent_data, **wt_converted}
            mixed_units_count += 1
        
        # Store the converted composition
        curve_info['alloy_composition_converted'] = converted_composition
        
    except Exception as e:
        parse_errors.append({
            'curve_id': curve_info.get('curve_id'),
            'error': str(e),
            'composition': composition
        })
        curve_info['alloy_composition_converted'] = {}

print(f"✓ Converted from wt% to at%: {converted_count} curves")
print(f"✓ Already in at%: {already_at_count} curves")
print(f"⚠️  Mixed units (converted wt% portion): {mixed_units_count} curves")

print(f"\nTotal curves processed: {len(curves_list)}")



🔬 Step 5: Converting wt% to at%
✓ Converted from wt% to at%: 382 curves
✓ Already in at%: 0 curves
⚠️  Mixed units (converted wt% portion): 0 curves

Total curves processed: 392


In [127]:
print("\n" + "="*60)
print("STEP 5: Creating Composition String Target")
print("="*60)

def composition_to_string(composition_dict):
    """
    Convert composition dictionary to standardized string.
    Elements are sorted alphabetically and concatenated as "ElementPercent".
    
    Example: {"Fe":49, "Mn":30, "Co":10, "Cr":10, "N":1} 
    Returns: "Co10-Cr10-Fe49-Mn30-N1"
    
    Rules:
    - Alphabetically sorted by element name
    - Format: Element + rounded percentage
    - Separated by hyphens
    - Skip elements with None or 0 values
    """
    if not composition_dict:
        return ""
    
    # Filter out None and 0 values, convert to numeric
    valid_elements = {}
    for element, value in composition_dict.items():
        # Try to convert value to float
        try:
            if value is not None:
                numeric_value = float(value)
                if numeric_value > 0:
                    # Round to 2 decimal places
                    rounded = round(numeric_value, 2)
                    # Remove .0 if it's a whole number
                    if rounded == int(rounded):
                        valid_elements[element] = int(rounded)
                    else:
                        valid_elements[element] = rounded
        except (ValueError, TypeError):
            # Skip values that can't be converted to float
            continue
    
    # Sort alphabetically and create string
    sorted_elements = sorted(valid_elements.items())
    
    # Create string like "Co10-Cr10-Fe49-Mn30-N1"
    composition_string = "-".join([f"{elem}{val}" for elem, val in sorted_elements])
    
    return composition_string

# Create composition strings for all curves
curves_with_composition = 0
curves_without_composition = 0

for curve_info in curves_list:
    composition = curve_info.get('alloy_composition', {})
    
    if composition:
        comp_string = composition_to_string(composition)
        curve_info['composition_string'] = comp_string
        if comp_string:
            curves_with_composition += 1
        else:
            curves_without_composition += 1
    else:
        curve_info['composition_string'] = ""
        curves_without_composition += 1

print(f"✓ Curves with composition string: {curves_with_composition}")
print(f"✓ Curves without composition: {curves_without_composition}")

# Analyze unique compositions
unique_compositions = set()
for curve_info in curves_list:
    comp_string = curve_info.get('composition_string', '')
    if comp_string:
        unique_compositions.add(comp_string)

print(f"\n📊 Statistics:")
print(f"  Total unique alloy compositions: {len(unique_compositions)}")
print(f"  Total curves processed: {len(curves_list)}")

# Show first 10 examples
print("\n" + "="*60)
print("📋 Sample Composition Strings (First 10):")
print("="*60)

for i, curve_info in enumerate(curves_list[:10]):
    comp_dict = curve_info.get('alloy_composition', {})
    comp_string = curve_info.get('composition_string', '')
    
    print(f"\n{i+1}. Curve ID: {curve_info['curve_id']}")
    print(f"   Dict: {comp_dict}")
    print(f"   String: {comp_string}")

# Show some unique compositions
print("\n" + "="*60)
print("🔬 Sample of Unique Compositions (First 20):")
print("="*60)
for i, comp in enumerate(sorted(list(unique_compositions))[:20]):
    print(f"{i+1}. {comp}")

print("\n" + "="*60)
print("✅ PREPROCESSING COMPLETE!")
print("="*60)
print(f"Total curves: {len(curves_list)}")
print(f"Unique compositions: {len(unique_compositions)}")
print(f"Curves with target string: {curves_with_composition}")
print("\nYour target variable 'composition_string' is ready with NO NaN values!")
print("Format: 'Co10-Cr10-Fe49-Mn30-N1' (alphabetically sorted)")


STEP 5: Creating Composition String Target
✓ Curves with composition string: 382
✓ Curves without composition: 10

📊 Statistics:
  Total unique alloy compositions: 77
  Total curves processed: 392

📋 Sample Composition Strings (First 10):

1. Curve ID: fig14a-77K
   Dict: {'Co': 20.0, 'Cr': 20.0, 'Fe': 20.0, 'Mn': 20.0, 'Ni': 20.0}
   String: Co20-Cr20-Fe20-Mn20-Ni20

2. Curve ID: fig14a-293K
   Dict: {'Co': 20.0, 'Cr': 20.0, 'Fe': 20.0, 'Mn': 20.0, 'Ni': 20.0}
   String: Co20-Cr20-Fe20-Mn20-Ni20

3. Curve ID: fig14a-473K
   Dict: {'Co': 20.0, 'Cr': 20.0, 'Fe': 20.0, 'Mn': 20.0, 'Ni': 20.0}
   String: Co20-Cr20-Fe20-Mn20-Ni20

4. Curve ID: fig14a-673K
   Dict: {'Co': 20.0, 'Cr': 20.0, 'Fe': 20.0, 'Mn': 20.0, 'Ni': 20.0}
   String: Co20-Cr20-Fe20-Mn20-Ni20

5. Curve ID: fig14a-873K
   Dict: {'Co': 20.0, 'Cr': 20.0, 'Fe': 20.0, 'Mn': 20.0, 'Ni': 20.0}
   String: Co20-Cr20-Fe20-Mn20-Ni20

6. Curve ID: fig14a-1073K
   Dict: {'Co': 20.0, 'Cr': 20.0, 'Fe': 20.0, 'Mn': 20.0, 'Ni': 20.0}
   S

In [100]:
# STEP 6A: Define feature extraction function
# This function extracts 54 comprehensive features from stress-strain curve data

import numpy as np
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter
from scipy.integrate import trapezoid

def extract_features_from_curve(curve_data, debug=False):
    """
    Extract COMPREHENSIVE features from a stress-strain curve.
    Returns 54 features covering mechanical properties, statistics, and curve shape.
    """
    # === STEP 1: Check if data is valid ===
    if not curve_data or len(curve_data) < 6:
        if debug:
            print("⚠️ Skipping: Not enough data points")
        return None
    
    # === STEP 2: Convert to arrays ===
    strains_unsorted = np.array([p['x'] for p in curve_data])
    stresses_unsorted = np.array([p['y'] for p in curve_data])
    
    # === STEP 3: Sort by strain ===
    sort_idx = np.argsort(strains_unsorted)
    strains = strains_unsorted[sort_idx]
    stresses = stresses_unsorted[sort_idx]
    
    # Initialize features dictionary
    features = {}
    
    # === STEP 4: Basic Mechanical Properties ===
    features['ultimate_tensile_strength'] = float(np.max(stresses))
    features['max_strain'] = float(np.max(strains))
    features['uts_strain'] = float(strains[np.argmax(stresses)])
    features['min_stress'] = float(np.min(stresses))
    features['min_strain'] = float(np.min(strains))
    
    # === STEP 5: Yield Strength (0.2% offset approximation) ===
    if strains[0] <= 0.002 <= strains[-1]:
        f_interp = interp1d(strains, stresses, kind='linear', fill_value='extrapolate', bounds_error=False)
        features['yield_strength_002'] = float(f_interp(0.002))
    else:
        features['yield_strength_002'] = float(stresses[0])
    
    features['yield_to_uts_ratio'] = float(features['yield_strength_002'] / (features['ultimate_tensile_strength'] + 1e-10))
    
    # === STEP 6: Stress at specific strain points ===
    strain_points = [0.01, 0.02, 0.05, 0.1, 0.2, 0.3, 0.5]
    for sp in strain_points:
        if strains[0] <= sp <= strains[-1]:
            f_interp = interp1d(strains, stresses, kind='linear', fill_value='extrapolate', bounds_error=False)
            features[f'stress_at_{sp}'] = float(f_interp(sp))
        else:
            features[f'stress_at_{sp}'] = np.nan
    
    # === STEP 7: Elastic Modulus (Young's Modulus) ===
    elastic_portion = int(len(strains) * 0.2)
    if elastic_portion > 2:
        elastic_strains = strains[:elastic_portion]
        elastic_stresses = stresses[:elastic_portion]
        if len(elastic_strains) > 1 and (elastic_strains[-1] - elastic_strains[0]) > 1e-6:
            E_fit = np.polyfit(elastic_strains, elastic_stresses, 1)
            features['elastic_modulus'] = float(E_fit[0])
            features['elastic_intercept'] = float(E_fit[1])
        else:
            features['elastic_modulus'] = np.nan
            features['elastic_intercept'] = np.nan
    else:
        features['elastic_modulus'] = np.nan
        features['elastic_intercept'] = np.nan
    
    # === STEP 8: Strain Hardening Exponent (n-value) ===
    plastic_mask = strains > 0.002
    if np.sum(plastic_mask) > 5:
        plastic_strains = strains[plastic_mask]
        plastic_stresses = stresses[plastic_mask]
        valid_mask = (plastic_strains > 0) & (plastic_stresses > 0)
        if np.sum(valid_mask) > 3:
            try:
                log_strain = np.log(plastic_strains[valid_mask])
                log_stress = np.log(plastic_stresses[valid_mask])
                n_fit = np.polyfit(log_strain, log_stress, 1)
                features['strain_hardening_exponent'] = float(n_fit[0])
                features['strength_coefficient_K'] = float(np.exp(n_fit[1]))
            except:
                features['strain_hardening_exponent'] = np.nan
                features['strength_coefficient_K'] = np.nan
        else:
            features['strain_hardening_exponent'] = np.nan
            features['strength_coefficient_K'] = np.nan
    else:
        features['strain_hardening_exponent'] = np.nan
        features['strength_coefficient_K'] = np.nan
    
    # === STEP 9: Work Hardening Analysis ===
    if len(strains) > 5:
        window = min(5, len(stresses) if len(stresses) % 2 == 1 else len(stresses) - 1)
        if window >= 3:
            stresses_smooth = savgol_filter(stresses, window, 2)
        else:
            stresses_smooth = stresses
        
        d_stress = np.diff(stresses_smooth)
        d_strain = np.diff(strains)
        hardening_rate = d_stress / (d_strain + 1e-10)
        
        features['avg_hardening_rate'] = float(np.mean(hardening_rate))
        features['max_hardening_rate'] = float(np.max(hardening_rate))
        features['min_hardening_rate'] = float(np.min(hardening_rate))
        features['std_hardening_rate'] = float(np.std(hardening_rate))
        
        n_points = len(hardening_rate)
        if n_points >= 3:
            features['hardening_rate_early'] = float(np.mean(hardening_rate[:n_points//3]))
            features['hardening_rate_mid'] = float(np.mean(hardening_rate[n_points//3:2*n_points//3]))
            features['hardening_rate_late'] = float(np.mean(hardening_rate[2*n_points//3:]))
        else:
            features['hardening_rate_early'] = float(np.mean(hardening_rate))
            features['hardening_rate_mid'] = float(np.mean(hardening_rate))
            features['hardening_rate_late'] = float(np.mean(hardening_rate))
        
        features['hardening_rate_ratio_early_mid'] = float(features['hardening_rate_early'] / (features['hardening_rate_mid'] + 1e-10))
        features['hardening_rate_ratio_mid_late'] = float(features['hardening_rate_mid'] / (features['hardening_rate_late'] + 1e-10))
    
    # === STEP 10: Energy Metrics ===
    features['toughness'] = float(trapezoid(stresses, strains))
    elastic_idx = int(len(strains) * 0.2)
    if elastic_idx > 1:
        features['resilience'] = float(trapezoid(stresses[:elastic_idx], strains[:elastic_idx]))
    else:
        features['resilience'] = 0.0
    
    features['energy_elastic_region'] = float(trapezoid(stresses[:elastic_idx], strains[:elastic_idx]) if elastic_idx > 1 else 0)
    features['energy_plastic_region'] = float(trapezoid(stresses[elastic_idx:], strains[elastic_idx:]) if len(strains) > elastic_idx else 0)
    features['plastic_to_elastic_energy'] = float(features['energy_plastic_region'] / (features['energy_elastic_region'] + 1e-10))
    
    # === STEP 11: Statistical Features ===
    features['stress_mean'] = float(np.mean(stresses))
    features['stress_std'] = float(np.std(stresses))
    features['stress_median'] = float(np.median(stresses))
    features['stress_25_percentile'] = float(np.percentile(stresses, 25))
    features['stress_75_percentile'] = float(np.percentile(stresses, 75))
    features['stress_range'] = float(np.max(stresses) - np.min(stresses))
    features['stress_cv'] = float(features['stress_std'] / (features['stress_mean'] + 1e-10))
    
    features['strain_mean'] = float(np.mean(strains))
    features['strain_std'] = float(np.std(strains))
    
    # === STEP 12: Curve Shape Features ===
    features['num_data_points'] = len(strains)
    features['strain_range'] = float(np.max(strains) - np.min(strains))
    
    n = len(strains)
    if n >= 4:
        early_idx = max(1, n // 4)
        early_slope = (stresses[early_idx] - stresses[0]) / (strains[early_idx] - strains[0] + 1e-10)
        features['early_slope'] = float(early_slope)
        
        mid_idx = n // 2
        mid_slope = (stresses[mid_idx] - stresses[early_idx]) / (strains[mid_idx] - strains[early_idx] + 1e-10)
        features['mid_slope'] = float(mid_slope)
        
        late_slope = (stresses[-1] - stresses[mid_idx]) / (strains[-1] - strains[mid_idx] + 1e-10)
        features['late_slope'] = float(late_slope)
        
        features['slope_ratio_early_mid'] = float(early_slope / (mid_slope + 1e-10))
        features['slope_ratio_mid_late'] = float(mid_slope / (late_slope + 1e-10))
    else:
        features['early_slope'] = np.nan
        features['mid_slope'] = np.nan
        features['late_slope'] = np.nan
        features['slope_ratio_early_mid'] = np.nan
        features['slope_ratio_mid_late'] = np.nan
    
    if n > 2:
        second_deriv = np.diff(np.diff(stresses)) / (np.diff(strains[:-1]) + 1e-10) ** 2
        features['avg_curvature'] = float(np.mean(second_deriv))
        features['max_curvature'] = float(np.max(np.abs(second_deriv)))
    else:
        features['avg_curvature'] = np.nan
        features['max_curvature'] = np.nan
    
    # === STEP 13: Stress Ratios ===
    if 'stress_at_0.1' in features and 'stress_at_0.01' in features and not np.isnan(features['stress_at_0.1']) and not np.isnan(features['stress_at_0.01']):
        features['stress_ratio_0.1_to_0.01'] = float(features['stress_at_0.1'] / (features['stress_at_0.01'] + 1e-10))
    else:
        features['stress_ratio_0.1_to_0.01'] = np.nan
    
    if 'stress_at_0.2' in features and 'stress_at_0.1' in features and not np.isnan(features['stress_at_0.2']) and not np.isnan(features['stress_at_0.1']):
        features['stress_ratio_0.2_to_0.1'] = float(features['stress_at_0.2'] / (features['stress_at_0.1'] + 1e-10))
    else:
        features['stress_ratio_0.2_to_0.1'] = np.nan
    
    # === STEP 14: Detect True vs Engineering Stress-Strain ===
    last_portion = int(len(stresses) * 0.8)
    if last_portion < len(stresses):
        last_stress_avg = np.mean(stresses[last_portion:])
        peak_stress = np.max(stresses[:last_portion])
        stress_drop_ratio = (peak_stress - last_stress_avg) / (peak_stress + 1e-10)
        features['is_likely_engineering'] = 1.0 if stress_drop_ratio > 0.05 else 0.0
        features['stress_drop_ratio'] = float(stress_drop_ratio)
    else:
        features['is_likely_engineering'] = 0.0
        features['stress_drop_ratio'] = 0.0
    
    return features

print("✓ Feature extraction function defined! (54 features)")
print("  Ready to extract features from curve_data in next step.")



✓ Feature extraction function defined! (54 features)
  Ready to extract features from curve_data in next step.


In [129]:
# =============================================================================
# STEP 6B: Create DataFrame with features + composition string target
# =============================================================================

import pandas as pd

print("\n" + "="*60)
print("STEP 6B: Creating DataFrame with Features + Target")
print("="*60)

# Prepare the data for DataFrame creation
df_data = []
skipped_count = 0

for curve_info in curves_list:
    # Extract 54 features from curve_data
    curve_data = curve_info.get('curve_data', [])
    extracted_features = extract_features_from_curve(curve_data, debug=False)
    
    if extracted_features is None:
        skipped_count += 1
        continue
    
    row = {
        # Metadata (for tracking only, not used as features)
        'paper_title': curve_info.get('paper_title', ''),
        'graph_id': curve_info.get('graph_id', ''),
        'curve_id': curve_info.get('curve_id', ''),
        'curve_label': curve_info.get('curve_label', ''),
        
        # Axis labels and units (for reference)
        'x_axis_label': curve_info.get('x_axis_label', ''),
        'y_axis_label': curve_info.get('y_axis_label', ''),
        'y_unit': curve_info.get('y_unit', ''),
        
        # TARGET VARIABLE: Composition String
        'composition_string': curve_info.get('composition_string', ''),
    }
    
    # Add all 54 extracted features to the row
    row.update(extracted_features)
    
    # Expand alloy composition dictionary into separate columns (for reference/analysis)
    alloy_composition = curve_info.get('alloy_composition', {})
    for element, percentage in alloy_composition.items():
        # Clean element name (remove any units or extra info)
        element_clean = element.split('(')[0].strip() if '(' in element else element.strip()
        row[element_clean] = percentage  # Simple column name: Al, Cr, Fe, etc.
    
    df_data.append(row)

# Create DataFrame
df = pd.DataFrame(df_data)

print(f"✓ DataFrame shape: {df.shape} (rows: {df.shape[0]}, columns: {df.shape[1]})")
print(f"✓ Successfully processed: {len(df)} curves")
if skipped_count > 0:
    print(f"⚠️  Skipped {skipped_count} curves (insufficient data points)")

# Identify column categories
metadata_cols = ['paper_title', 'graph_id', 'curve_id', 'curve_label']
axis_cols = ['x_axis_label', 'y_axis_label', 'y_unit']
target_col = 'composition_string'

common_elements = ['Al', 'B', 'C', 'Co', 'Cr', 'Cu', 'Fe', 'Hf', 'Mg', 'Mn', 'Mo', 
                   'N', 'Nb', 'Nd', 'Ni', 'Si', 'Ta', 'Ti', 'V', 'W', 'Y', 'Zr']
alloy_cols = [c for c in df.columns if c in common_elements]

# The 54 extracted features are all other columns
all_cols = set(df.columns)
excluded_cols = set(metadata_cols + axis_cols + alloy_cols + [target_col])
feature_cols = sorted([c for c in all_cols if c not in excluded_cols])

print(f"\n📋 Column Categories:")
print(f"  🎯 TARGET: {target_col}")
print(f"  📊 Extracted Features: {len(feature_cols)}")
print(f"  📝 Metadata (tracking): {len(metadata_cols)}")
print(f"  🔬 Axis/Unit (reference): {len(axis_cols)}")
print(f"  ⚗️  Alloy composition (reference): {len(alloy_cols)}")

print(f"\n📊 Feature Categories ({len(feature_cols)} total):")
print(f"  • Basic Mechanical Properties")
print(f"  • Yield Strength")
print(f"  • Stress at Strain Points")
print(f"  • Elastic Properties")
print(f"  • Strain Hardening")
print(f"  • Work Hardening")
print(f"  • Energy Metrics")
print(f"  • Statistical")
print(f"  • Curve Shape")
print(f"  • Stress Ratios")
print(f"  • Engineering Detection")

print(f"\n📈 Sample Feature Names (first 10):")
for i, feat in enumerate(feature_cols[:10], 1):
    print(f"  {i:2d}. {feat}")

# Check target variable
print(f"\n🎯 Target Variable Analysis:")
print(f"  Column: {target_col}")
print(f"  Non-empty targets: {df[target_col].notna().sum()}")
print(f"  Empty targets: {df[target_col].isna().sum() + (df[target_col] == '').sum()}")
print(f"  Unique compositions: {df[target_col].nunique()}")

print(f"\n📋 Sample Target Values (first 5):")
for i, comp_str in enumerate(df[target_col].head(5), 1):
    print(f"  {i}. {comp_str}")

# Preview DataFrame
print(f"\n📈 DataFrame Preview:")
preview_cols = ['curve_id', target_col] + feature_cols[:5]
print(df[preview_cols].head())



print(f"\n✅ DataFrame ready for machine learning!")
print(f"   ✓ Features (X): {len(feature_cols)} columns")
print(f"   ✓ Target (y): '{target_col}' - composition string")



STEP 6B: Creating DataFrame with Features + Target
✓ DataFrame shape: (392, 84) (rows: 392, columns: 84)
✓ Successfully processed: 392 curves

📋 Column Categories:
  🎯 TARGET: composition_string
  📊 Extracted Features: 54
  📝 Metadata (tracking): 4
  🔬 Axis/Unit (reference): 3
  ⚗️  Alloy composition (reference): 22

📊 Feature Categories (54 total):
  • Basic Mechanical Properties
  • Yield Strength
  • Stress at Strain Points
  • Elastic Properties
  • Strain Hardening
  • Work Hardening
  • Energy Metrics
  • Statistical
  • Curve Shape
  • Stress Ratios
  • Engineering Detection

📈 Sample Feature Names (first 10):
   1. avg_curvature
   2. avg_hardening_rate
   3. early_slope
   4. elastic_intercept
   5. elastic_modulus
   6. energy_elastic_region
   7. energy_plastic_region
   8. hardening_rate_early
   9. hardening_rate_late
  10. hardening_rate_mid

🎯 Target Variable Analysis:
  Column: composition_string
  Non-empty targets: 392
  Empty targets: 10
  Unique compositions: 78

📋

/opt/homebrew/lib/python3.11/site-packages/scipy/interpolate/_interpolate.py:479: RuntimeWarning: divide by zero encountered in divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]
/opt/homebrew/lib/python3.11/site-packages/scipy/interpolate/_interpolate.py:482: RuntimeWarning: invalid value encountered in multiply
  y_new = slope*(x_new - x_lo)[:, None] + y_lo


In [130]:
# Check for missing values in features
missing_features = df[feature_cols].isna().sum()
features_with_missing = missing_features[missing_features > 0]
if len(features_with_missing) > 0:
    print(f"\n⚠️  Features with missing values:")
    for feat, count in features_with_missing.items():
        print(f"  {feat}: {count} missing ({count/len(df)*100:.1f}%)")
else:
    print(f"\n✓ No missing values in feature columns!")


⚠️  Features with missing values:
  elastic_intercept: 14 missing (3.6%)
  elastic_modulus: 14 missing (3.6%)
  stress_at_0.01: 141 missing (36.0%)
  stress_at_0.02: 73 missing (18.6%)
  stress_at_0.05: 42 missing (10.7%)
  stress_at_0.1: 59 missing (15.1%)
  stress_at_0.2: 136 missing (34.7%)
  stress_at_0.3: 213 missing (54.3%)
  stress_at_0.5: 314 missing (80.1%)
  stress_ratio_0.1_to_0.01: 184 missing (46.9%)
  stress_ratio_0.2_to_0.1: 141 missing (36.0%)
  yield_strength_002: 9 missing (2.3%)
  yield_to_uts_ratio: 9 missing (2.3%)


In [131]:
# STEP 7: Handle null/missing values and prepare for ML
# Fill null values with median, then prepare features and targets

print("="*80)
print("STEP 7: Data Preprocessing for Machine Learning")
print("="*80)


print("\n" + "="*80)
print("Step 7.1: Handling Null Values")
print("="*80)

null_counts = df[feature_cols].isnull().sum()
features_with_nulls = null_counts[null_counts > 0].sort_values(ascending=False)

if len(features_with_nulls) > 0:
    print(f"⚠️  Found {len(features_with_nulls)} features with null values:")
    for feature, count in features_with_nulls.items():
        percentage = (count / len(df)) * 100
        print(f"  {feature:35s}: {count:3d} nulls ({percentage:5.2f}%)")
    
    print("\n📊 Filling null values with median...")
    filled_count = 0
    for feature in features_with_nulls.index:
        median_value = df[feature].median()
        null_count = df[feature].isnull().sum()
        df[feature].fillna(median_value, inplace=True)
        filled_count += null_count
        print(f"  ✓ {feature:35s}: Filled {null_count:3d} nulls (median = {median_value:12.4f})")
    
    print(f"\n✅ Filled {filled_count} total null values")
else:
    print("✅ No null values found in feature columns!")




STEP 7: Data Preprocessing for Machine Learning

Step 7.1: Handling Null Values
⚠️  Found 13 features with null values:
  stress_at_0.5                      : 314 nulls (80.10%)
  stress_at_0.3                      : 213 nulls (54.34%)
  stress_ratio_0.1_to_0.01           : 184 nulls (46.94%)
  stress_at_0.01                     : 141 nulls (35.97%)
  stress_ratio_0.2_to_0.1            : 141 nulls (35.97%)
  stress_at_0.2                      : 136 nulls (34.69%)
  stress_at_0.02                     :  73 nulls (18.62%)
  stress_at_0.1                      :  59 nulls (15.05%)
  stress_at_0.05                     :  42 nulls (10.71%)
  elastic_intercept                  :  14 nulls ( 3.57%)
  elastic_modulus                    :  14 nulls ( 3.57%)
  yield_strength_002                 :   9 nulls ( 2.30%)
  yield_to_uts_ratio                 :   9 nulls ( 2.30%)

📊 Filling null values with median...
  ✓ stress_at_0.5                      : Filled 314 nulls (median =     725.0168)
  ✓ st

/var/folders/xy/0xhnpbcd2fsby1z0rgb21v_40000gn/T/ipykernel_41573/920556588.py:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[feature].fillna(median_value, inplace=True)
/var/folders/xy/0xhnpbcd2fsby1z0rgb21v_40000gn/T/ipykernel_41573/920556588.py:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always

In [132]:
# =============================================================================
# STEP 8: Separate Features (X) and Target (y)
# =============================================================================

print("\n" + "="*60)
print("STEP 8: Separating Features (X) and Target (y)")
print("="*60)

# Define X (features) and y (target)
X = df[feature_cols].copy()
y = df['composition_string'].copy()

print(f"✓ Features (X):")
print(f"  Shape: {X.shape}")
print(f"  Columns: {len(feature_cols)} features")
print(f"  Data type: {X.dtypes.value_counts().to_dict()}")

print(f"\n✓ Target (y):")
print(f"  Shape: {y.shape}")
print(f"  Column: 'composition_string'")
print(f"  Unique values: {y.nunique()}")
print(f"  Non-empty: {(y != '').sum()}")
print(f"  Empty: {(y == '').sum()}")

# Check for any remaining issues
print(f"\n🔍 Data Quality Check:")
print(f"  Missing values in X: {X.isnull().sum().sum()}")
print(f"  Missing values in y: {y.isnull().sum()}")
print(f"  Empty strings in y: {(y == '').sum()}")
print(f"  Infinite values in X: {np.isinf(X.values).sum()}")

# Remove rows with empty composition strings (if any)
valid_mask = (y != '') & (y.notna())
if valid_mask.sum() < len(y):
    removed = len(y) - valid_mask.sum()
    print(f"\n⚠️  Removing {removed} rows with empty/null target values...")
    X = X[valid_mask].copy()
    y = y[valid_mask].copy()
    print(f"  New X shape: {X.shape}")
    print(f"  New y shape: {y.shape}")

# Show sample data
print(f"\n📊 Sample Data:")
print(f"\nFirst 5 rows of X (features):")
print(X.head())

print(f"\nFirst 10 target values (y):")
for i, comp in enumerate(y.head(10), 1):
    print(f"  {i:2d}. {comp}")

# Summary statistics
print(f"\n📈 Feature Statistics:")
print(X.describe().T[['mean', 'std', 'min', 'max']].head(10))

print(f"\n✅ Data ready for machine learning!")
print(f"   X (features): {X.shape[0]} samples × {X.shape[1]} features")
print(f"   y (target): {y.shape[0]} composition strings")
print(f"   Ready for train/test split!")


STEP 8: Separating Features (X) and Target (y)
✓ Features (X):
  Shape: (392, 54)
  Columns: 54 features
  Data type: {dtype('float64'): 53, dtype('int64'): 1}

✓ Target (y):
  Shape: (392,)
  Column: 'composition_string'
  Unique values: 78
  Non-empty: 382
  Empty: 10

🔍 Data Quality Check:
  Missing values in X: 0
  Missing values in y: 0
  Empty strings in y: 10
  Infinite values in X: 0

⚠️  Removing 10 rows with empty/null target values...
  New X shape: (382, 54)
  New y shape: (382,)

📊 Sample Data:

First 5 rows of X (features):
   avg_curvature  avg_hardening_rate  early_slope  elastic_intercept  \
0   -3920.515359          657.856710  1494.701298         585.781808   
1   -5964.619605          360.233306  1411.249999         350.447240   
2   -6369.203602          342.997404  1285.157893         287.743913   
3   -5994.481358          449.550470  1240.103894         261.107046   
4   -3222.726944          247.017432   989.869564         246.236152   

   elastic_modulus  en

In [133]:
# =============================================================================
# STEP 9: Train/Test Split (80/20)
# =============================================================================

from sklearn.model_selection import train_test_split

print("\n" + "="*60)
print("STEP 9: Train/Test Split (80/20)")
print("="*60)

# Split the data: 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    shuffle=True
)

print(f"✓ Train/Test Split Complete:")
print(f"\n📊 Training Set:")
print(f"  X_train shape: {X_train.shape}")
print(f"  y_train shape: {y_train.shape}")
print(f"  Samples: {len(X_train)} ({len(X_train)/len(X)*100:.1f}%)")
print(f"  Unique compositions: {y_train.nunique()}")

print(f"\n📊 Test Set:")
print(f"  X_test shape: {X_test.shape}")
print(f"  y_test shape: {y_test.shape}")
print(f"  Samples: {len(X_test)} ({len(X_test)/len(X)*100:.1f}%)")
print(f"  Unique compositions: {y_test.nunique()}")

# # Check for data leakage (compositions in test that aren't in train)
# train_compositions = set(y_train.unique())
# test_compositions = set(y_test.unique())
# new_compositions_in_test = test_compositions - train_compositions

# print(f"\n🔍 Composition Distribution:")
# print(f"  Compositions in training: {len(train_compositions)}")
# print(f"  Compositions in test: {len(test_compositions)}")
# print(f"  New compositions in test (not in train): {len(new_compositions_in_test)}")

# if len(new_compositions_in_test) > 0:
#     print(f"\n⚠️  Warning: {len(new_compositions_in_test)} compositions appear in test but not in training")
#     print(f"  This is expected for compositional prediction tasks")
#     if len(new_compositions_in_test) <= 10:
#         print(f"  Examples:")
#         for comp in list(new_compositions_in_test)[:10]:
#             print(f"    • {comp}")

# Show sample from training set
print(f"\n📋 Sample Training Data:")
print(f"\nFirst 5 training samples (features):")
print(X_train.head())

print(f"\nFirst 10 training targets:")
for i, (idx, comp) in enumerate(y_train.head(10).items(), 1):
    print(f"  {i:2d}. {comp}")

# Show sample from test set
print(f"\n📋 Sample Test Data:")
print(f"\nFirst 5 test targets:")
for i, (idx, comp) in enumerate(y_test.head(5).items(), 1):
    print(f"  {i:2d}. {comp}")

print(f"\n✅ Data split complete and ready for model training!")
print(f"   Training: {len(X_train)} samples")
print(f"   Testing: {len(X_test)} samples")
print(f"   Features: {X_train.shape[1]}")


STEP 9: Train/Test Split (80/20)
✓ Train/Test Split Complete:

📊 Training Set:
  X_train shape: (305, 54)
  y_train shape: (305,)
  Samples: 305 (79.8%)
  Unique compositions: 74

📊 Test Set:
  X_test shape: (77, 54)
  y_test shape: (77,)
  Samples: 77 (20.2%)
  Unique compositions: 32

📋 Sample Training Data:

First 5 training samples (features):
     avg_curvature  avg_hardening_rate   early_slope  elastic_intercept  \
171  -2.013761e+04        2.257324e+03   4931.857131         537.003084   
330   4.240000e+19        2.565817e+10  24347.999513         255.013316   
225  -3.891944e+19        1.563723e+10   5665.826659         472.228329   
100   8.116667e+18        1.094302e+10  20711.230610          76.243714   
237   1.325250e+20        6.706886e+10  85230.495738          32.050000   

     elastic_modulus  energy_elastic_region  energy_plastic_region  \
171      5306.424741              15.337696             184.448393   
330     23526.263158               1.442143              4

In [134]:
# =============================================================================
# STEP 10: Feature Scaling using StandardScaler
# =============================================================================

from sklearn.preprocessing import StandardScaler

print("\n" + "="*60)
print("STEP 10: Feature Scaling (StandardScaler)")
print("="*60)

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler on training data and transform both train and test
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame to preserve column names
X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_cols, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_cols, index=X_test.index)

print(f"✓ Scaling Complete:")
print(f"\n📊 Training Set (Scaled):")
print(f"  Shape: {X_train_scaled.shape}")
print(f"  Mean: ~{X_train_scaled.mean().mean():.6f} (should be ~0)")
print(f"  Std: ~{X_train_scaled.std().mean():.6f} (should be ~1)")

print(f"\n📊 Test Set (Scaled):")
print(f"  Shape: {X_test_scaled.shape}")
print(f"  Mean: ~{X_test_scaled.mean().mean():.6f}")
print(f"  Std: ~{X_test_scaled.std().mean():.6f}")

# Show before/after comparison for first feature
sample_feature = feature_cols[0]
print(f"\n📈 Before/After Comparison (feature: '{sample_feature}'):")
print(f"\n  BEFORE scaling (training set):")
print(f"    Mean: {X_train[sample_feature].mean():.4f}")
print(f"    Std:  {X_train[sample_feature].std():.4f}")
print(f"    Min:  {X_train[sample_feature].min():.4f}")
print(f"    Max:  {X_train[sample_feature].max():.4f}")

print(f"\n  AFTER scaling (training set):")
print(f"    Mean: {X_train_scaled[sample_feature].mean():.4f}")
print(f"    Std:  {X_train_scaled[sample_feature].std():.4f}")
print(f"    Min:  {X_train_scaled[sample_feature].min():.4f}")
print(f"    Max:  {X_train_scaled[sample_feature].max():.4f}")

# Show sample of scaled data
print(f"\n📋 Sample Scaled Training Data (first 5 rows, first 5 features):")
print(X_train_scaled[feature_cols[:5]].head())

# Verify no missing values after scaling
print(f"\n🔍 Data Quality Check After Scaling:")
print(f"  Missing values in X_train_scaled: {X_train_scaled.isnull().sum().sum()}")
print(f"  Missing values in X_test_scaled: {X_test_scaled.isnull().sum().sum()}")
print(f"  Infinite values in X_train_scaled: {np.isinf(X_train_scaled.values).sum()}")
print(f"  Infinite values in X_test_scaled: {np.isinf(X_test_scaled.values).sum()}")

# Statistics for all features after scaling
print(f"\n📊 Scaled Feature Statistics (Training Set):")
scaling_stats = pd.DataFrame({
    'mean': X_train_scaled.mean(),
    'std': X_train_scaled.std(),
    'min': X_train_scaled.min(),
    'max': X_train_scaled.max()
})
print(scaling_stats.head(10))

print(f"\n✅ Feature scaling complete!")
print(f"   ✓ X_train_scaled: {X_train_scaled.shape}")
print(f"   ✓ X_test_scaled: {X_test_scaled.shape}")
print(f"   ✓ All features normalized (mean≈0, std≈1)")
print(f"   ✓ Ready for model training!")


STEP 10: Feature Scaling (StandardScaler)
✓ Scaling Complete:

📊 Training Set (Scaled):
  Shape: (305, 54)
  Mean: ~-0.000000 (should be ~0)
  Std: ~1.001643 (should be ~1)

📊 Test Set (Scaled):
  Shape: (77, 54)
  Mean: ~-0.064571
  Std: ~0.626254

📈 Before/After Comparison (feature: 'avg_curvature'):

  BEFORE scaling (training set):
    Mean: 10245509944497967104.0000
    Std:  343238415231287099392.0000
    Min:  -2649007142857117007872.0000
    Max:  2265076744186033078272.0000

  AFTER scaling (training set):
    Mean: 0.0000
    Std:  1.0016
    Min:  -7.7603
    Max:  6.5801

📋 Sample Scaled Training Data (first 5 rows, first 5 features):
     avg_curvature  avg_hardening_rate  early_slope  elastic_intercept  \
171      -0.029899           -0.317487    -0.057354           0.373383   
330       0.093834            0.216487    -0.057354           0.007293   
225      -0.143474            0.007941    -0.057354           0.289290   
100      -0.006212           -0.089751    -0.057

In [135]:
# =============================================================================
# STEP 11: Train Random Forest Classifier
# =============================================================================

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import time

print("\n" + "="*60)
print("STEP 11: Random Forest Classifier Training")
print("="*60)

# Initialize Random Forest Classifier
print("\n🌲 Initializing Random Forest Classifier...")
rf_model = RandomForestClassifier(
    n_estimators=100,        # Number of trees
    max_depth=None,          # No limit on depth
    min_samples_split=2,     # Minimum samples to split a node
    min_samples_leaf=1,      # Minimum samples at leaf node
    random_state=42,         # For reproducibility
    n_jobs=-1,               # Use all CPU cores
    verbose=1                # Show progress
)

print(f"\n📋 Model Configuration:")
print(f"  Number of trees: {rf_model.n_estimators}")
print(f"  Max depth: {rf_model.max_depth}")
print(f"  Min samples split: {rf_model.min_samples_split}")
print(f"  Min samples leaf: {rf_model.min_samples_leaf}")
print(f"  Random state: {rf_model.random_state}")

# Train the model
print(f"\n🚀 Training Random Forest on {len(X_train_scaled)} samples...")
print(f"   Features: {X_train_scaled.shape[1]}")
print(f"   Unique target classes: {y_train.nunique()}")

start_time = time.time()
rf_model.fit(X_train_scaled, y_train)
training_time = time.time() - start_time

print(f"\n✅ Training Complete!")
print(f"   Time taken: {training_time:.2f} seconds ({training_time/60:.2f} minutes)")

# Make predictions on training set
print(f"\n📊 Evaluating on Training Set...")
y_train_pred = rf_model.predict(X_train_scaled)
train_accuracy = accuracy_score(y_train, y_train_pred)

print(f"   Training Accuracy: {train_accuracy*100:.2f}%")
print(f"   Correct predictions: {(y_train == y_train_pred).sum()} / {len(y_train)}")

# Make predictions on test set
print(f"\n📊 Evaluating on Test Set...")
y_test_pred = rf_model.predict(X_test_scaled)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"   Test Accuracy: {test_accuracy*100:.2f}%")
print(f"   Correct predictions: {(y_test == y_test_pred).sum()} / {len(y_test)}")

# Show sample predictions
print(f"\n🔍 Sample Predictions (First 10 from Test Set):")
print(f"{'#':<3} {'Actual':<40} {'Predicted':<40} {'Match':<6}")
print("="*95)
for i, (actual, predicted) in enumerate(zip(y_test.head(10), y_test_pred[:10]), 1):
    match = "✓" if actual == predicted else "✗"
    print(f"{i:<3} {actual:<40} {predicted:<40} {match:<6}")

# Feature importance
print(f"\n📈 Top 10 Most Important Features:")
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

for i, row in feature_importance.head(10).iterrows():
    print(f"  {row['feature']:<35s}: {row['importance']:.6f}")

# Model summary
print(f"\n" + "="*60)
print("📊 Model Performance Summary")
print("="*60)
print(f"  Training Accuracy: {train_accuracy*100:.2f}%")
print(f"  Test Accuracy: {test_accuracy*100:.2f}%")
print(f"  Overfitting: {(train_accuracy - test_accuracy)*100:.2f}%")
print(f"  Training samples: {len(X_train_scaled)}")
print(f"  Test samples: {len(X_test_scaled)}")
print(f"  Total classes: {y_train.nunique()}")
print(f"  Training time: {training_time:.2f} seconds")

print(f"\n✅ Random Forest model trained successfully!")


STEP 11: Random Forest Classifier Training

🌲 Initializing Random Forest Classifier...

📋 Model Configuration:
  Number of trees: 100
  Max depth: None
  Min samples split: 2
  Min samples leaf: 1
  Random state: 42

🚀 Training Random Forest on 305 samples...
   Features: 54
   Unique target classes: 74


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s



✅ Training Complete!
   Time taken: 0.22 seconds (0.00 minutes)

📊 Evaluating on Training Set...
   Training Accuracy: 100.00%
   Correct predictions: 305 / 305

📊 Evaluating on Test Set...
   Test Accuracy: 45.45%
   Correct predictions: 35 / 77

🔍 Sample Predictions (First 10 from Test Set):
#   Actual                                   Predicted                                Match 
1   Mo20-Nb20-Ta20-V20-W20                   Mo20-Nb20-Ta20-V20-W20                   ✓     
2   Al9-Co18.2-Cr18.2-Cu18.2-Fe18.2-Ni18.2   Al9-Co18.2-Cr18.2-Cu18.2-Fe18.2-Ni18.2   ✓     
3   Co10-Cr15-Fe42-Mn28-Si5                  Co10-Cr15-Fe42-Mn28-Si5                  ✓     
4   Co23.26-Cr23.26-Fe23.26-Mo6.96-Ni23.26   Co19.1-Cr14.8-Fe41.8-Mn18.8-Si5.5        ✗     
5   Co20-Cr15-Cu1.5-Fe38.5-Mn20-Si5          Co10-Cr15-Fe42-Mn28-Si5                  ✗     
6   Al25-Nb25-Ti25-V25                       Al16.39-Co16.39-Cr16.39-Fe16.39-Ni34.43  ✗     
7   Co10-Cr10-Fe50-Mn30                      Co19.1-C

[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
